# Setup

In [81]:
import matplotlib.pyplot as plt
import matplotlib
%matplotlib qt
# matplotlib.backend
# matplotlib.validate_backend("GTK3Ag")

In [82]:
import numpy as np
import pickle
from tqdm.auto import tqdm as progressbar
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import cv2
import cv2.aruco as aruco
import common as co
from os.path import join
from calib import calibrate_charuco_local, load_board, load_coefficients, save_coefficients

## Load calib

In [128]:
data_path="/home/freitas/TCC/ender-laser-scanner/pics/scan1_raspi/"

In [129]:
import json

# Calib params
with open("/home/freitas/TCC/ender-laser-scanner/calib_data/calib.pkl", "rb") as f:
    calib = pickle.load(f)
mtx = calib["mtx"]
dist_original = calib["dist"]
dist = None
n = calib["n"]

R = np.load("/home/freitas/TCC/ender-laser-scanner/calib_data/cam_to_printer.npy")

board, aruco_dict = load_board("board.pkl")

print(board.getChessboardSize())
print(board.getMarkerLength())
print(board.getSquareLength())

(16, 16)
9.375
12.5


In [130]:
R

array([[ 0.9985865 , -0.02415544,  0.04734473, -0.20928927],
       [ 0.00277792, -0.86582552, -0.50033835, -0.92947292],
       [ 0.05307817,  0.49976263, -0.86453457,  0.62204345]])

In [131]:
def load(path):
    info = {}
    data = []
    with open(join(path,"points.json")) as f:
        info = json.loads(f.read())

    for fig, point in info.items():
        image =  cv2.cvtColor(cv2.imread(join(path, fig)), cv2.COLOR_BGR2RGB)
        image = cv2.undistort(image, mtx, dist_original, None, mtx)
        data.append( (point, image) )

    return data

In [132]:
data = load(data_path+"low_exp")

points = [d[0] for d in data]
images = [d[1] for d in data]

In [133]:
red_threshold = 20
def find_laser(image):
    blur = cv2.GaussianBlur(image, ksize=(5,5), sigmaX=1)
    centroids = co.column_centroids(co.red_contrast(blur),
                                    mask=co.red_contrast(blur) > red_threshold)
                                    
    return centroids

In [134]:
from matplotlib import animation

# First set up the figure, the axis, and the plot element we want to animate
fig = plt.figure()
# animation function.  This is called sequentially
imgplot = None
pointsplot = None

def init():
    global imgplot, pointsplot
    imgplot = plt.imshow(data[0][1], animated=True)
    pointsplot, = plt.plot(range(data[0][1].shape[0]), "r.", animated=True)
    return pointsplot, imgplot


def laser_points_img_gen(i):
    global imgplot, pointsplot
    # Find board and laser
    point, image =data[i]

    # blur = cv2.GaussianBlur(image, ksize=(5,5), sigmaX=1)

    im = image
    centroids = find_laser(image)
    # centroids = co.column_argmaxes(co.red_contrast(image), mask=co.red_contrast(image)>50)
    p_centroids = np.stack(list((i, p, 1) for i,p in enumerate(centroids) if not np.isnan(p))).T

    # Update anim
    imgplot.set_data(data[i][1])
    pointsplot.set_data(p_centroids[0], p_centroids[1])
    return pointsplot, imgplot

# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, laser_points_img_gen, init_func=init,
                               frames=len(data), interval=20, blit=True)

In [135]:
def ray_plane_intersect_vectorized(img_points, plane, cam_mtx):
	"""
	Calculate 3d-intersection between image rays defined by `img_ponts` and `cam_mtx`
	and a plane defined by the 4-vec `plane`
	@param img_points np.array with shape (3, N), N= number of points, homegeneous coords
	@param plane np.array with shape = (4,)
	@param cam_mtx np.array with shape = (3,3), intrinsic camera parameters
	@returns np.array with shape (4, N) with 3d points
	"""
	assert plane.shape == (4,)
	assert len(img_points.shape) == 2
	assert img_points.shape[0] == 3

	rays = np.linalg.inv(cam_mtx)@img_points
	p = plane[-1]
	n = plane[:3]
	out = (-p/(n@rays)) * rays
	out = np.vstack((out, np.ones((1, out.shape[-1])))) # homogeneous coords
	return out

In [138]:
def triangulate_pts(image):
    centroids = find_laser(image)
    p_centroids = np.stack(list((i, p, 1) for i,p in enumerate(centroids) if not np.isnan(p))).T

    pts_3d = ray_plane_intersect_vectorized(p_centroids, n, mtx)
    return pts_3d
def chess_plane(rvec, tvec):
	# Find p
	n_chessboard = np.array([0, 0, 1])
	M,_ = cv2.Rodrigues(rvec)
	n3 = M@n_chessboard
	p = -1 * n3@tvec
	n = np.hstack((n3,p))
	return n

In [139]:
W = np.eye(4)
Xlist = []
XcList=[]
for point, image  in data:
    # 3d pts in camera frame
    pts_3d = triangulate_pts(image)
    # Extrinsic params
    # rot, jac = cv2.Rodrigues(rvec)
    # W[:3, -1] = tvec.flatten()
    # W[:3,:3] = rot

    # Concat points
    XcList.append(pts_3d)
    # Xlist.append(np.linalg.inv(W)@pts_3d)
# X = np.hstack(Xlist)

## Plot

In [140]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import importlib
import common as co
importlib.reload(co)

<module 'common' from '/home/freitas/TCC/ender-laser-scanner/notebook/common.py'>

### Profiles

In [141]:
import matplotlib.ticker as plticker

fig, ax = co.plot3d(proj_type="ortho")
for i, x in enumerate(XcList):
    #x = x.copy()
    #x = x[:,x[2] < 110]
    ax.scatter(x[0], i + x[1], x[2], label=f"{i}")

#co.set_axes_equal(ax)
ax.set_xlabel("$X_c ~[mm]$")
ax.set_ylabel("$i_p + Y_c ~[mm]$")
ax.set_zlabel("$Z_c ~[mm]$")

ax.invert_zaxis()

#ax.set_ylim(-0.5, i)
#ax.set_zlim(90, 100)
#ax.set_xlim(-60, 0)

#ax.xaxis.set_major_locator(plticker.MultipleLocator(base=1.0))
# ax.yaxis.set_major_locator(plticker.MultipleLocator(base=1.0))
# ax.zaxis.set_major_locator(plticker.MultipleLocator(base=1.0))

ax.legend(loc="upper right", ncol=2, title="Perfil ($i_p$)").set_draggable(True)
plt.title("Visualização 3D dos Perfis")
fig.tight_layout()

In [101]:
for i, x in enumerate(XcList):
    #x = x[:,x[2] < 110]
    plt.plot(x[0], x[2], "--.", label=f"{i}")

plt.gca().invert_yaxis()

plt.xlabel(r"$X_c~ [mm]$")
plt.ylabel(r"$Z_c~ [mm]$")
ax.legend(loc="best", ncol=2, title="Perfil ($i_p$)")
plt.grid()
plt.title("Projeção dos Perfis em XZ")
plt.legend().set_draggable(True)
plt.tight_layout()

/tmp/ipykernel_12506/1391353283.py:13: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all axes decorations. 
  plt.tight_layout()


# Reconstruct with `cam_to_printer`

In [142]:
R = np.load("/home/freitas/TCC/ender-laser-scanner/calib_data/cam_to_printer.npy")
print(R)
p0 = np.array(data[0][0])
print(p0)

cam2printer = np.eye(4)
cam2printer[:3,:3] = R[:3,:3]
print(cam2printer)

[[ 0.9985865  -0.02415544  0.04734473 -0.20928927]
 [ 0.00277792 -0.86582552 -0.50033835 -0.92947292]
 [ 0.05307817  0.49976263 -0.86453457  0.62204345]]
[125.  90.  60.]
[[ 0.9985865  -0.02415544  0.04734473  0.        ]
 [ 0.00277792 -0.86582552 -0.50033835  0.        ]
 [ 0.05307817  0.49976263 -0.86453457  0.        ]
 [ 0.          0.          0.          1.        ]]


In [143]:
W = np.eye(4)
Xlist = []
XcList=[]

for point, image in data:
    # 3d pts in camera frame
    pts_3d = triangulate_pts(image)

    # Transformation from first point to this one (printer coords)
    # B = np.eye(4)
    # B[:3,3] = np.array(point) - p0

    A = np.eye(4)
    A[:3,3] = R[:3,:3]@(np.array(point) - p0)


    # Transformation from the first camera to this one
    # A = cam2printer@B@np.linalg.inv(cam2printer)

    # Concat points
    XcList.append(pts_3d)
    Xlist.append(A@pts_3d)
X = np.hstack(Xlist)

In [41]:
import matplotlib.ticker as plticker

fig, ax = co.plot3d(proj_type="ortho")
for i, x in enumerate(Xlist):
    #x = np.linalg.inv(cam2printer)@x
    ax.scatter(x[0], x[1], x[2], label=f"{i}")

#ax.set_ylim(-0.5, i)
#ax.set_zlim(90, 100)
#ax.set_xlim(-60, 0)

#ax.xaxis.set_major_locator(plticker.MultipleLocator(base=1.0))
#ax.yaxis.set_major_locator(plticker.MultipleLocator(base=1.0))
#ax.zaxis.set_major_locator(plticker.MultipleLocator(base=1.0))

fig.tight_layout()

In [148]:
import matplotlib.ticker as plticker

fig, ax = co.plot3d(proj_type="ortho")
X_printer = np.linalg.inv(cam2printer)@X
    #x = np.linalg.inv(cam2printer)@x
X_printer = X_printer[:,X_printer[2] > -200]
# X_printer = X_printer[:,X_printer[1] > 92]
# X_printer = X_printer[:,X_printer[1] < 108]
subsample = 10
ax.scatter(X_printer[0, ::subsample], X_printer[1, ::subsample], X_printer[2, ::subsample], label=f"{i}")

#ax.set_ylim(-0.5, i)
#ax.set_zlim(90, 100)
#ax.set_xlim(-60, 0)

#ax.xaxis.set_major_locator(plticker.MultipleLocator(base=1.0))
#ax.yaxis.set_major_locator(plticker.MultipleLocator(base=1.0))
#ax.zaxis.set_major_locator(plticker.MultipleLocator(base=1.0))

co.set_axes_equal(ax)

fig.tight_layout()

In [149]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
#region = X[2] > -9999
ax.plot_trisurf(X_printer[0,::10],X_printer[1,::10], X_printer[2,::10], color="gray")
co.set_axes_equal(ax)

In [150]:
plt.figure()
plt.scatter(X_printer[0], X_printer[2], s=2)

In [51]:
-92+101

9